**Importing Libraries**

In [ ]:
import requests # This Libraries help us to fetch data from API
import pandas as pd # For handling and analyzing data
import numpy as np # Use for mathematical Operations
from sklearn.model_selection import train_test_split # Spliting data into Trianing And Testing
from sklearn.metrics import mean_squared_error # For model evaluations
from sklearn.preprocessing import LabelEncoder # To convert categorical data into numerical data
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor # Mdelfor classification and regression
from datetime import datetime ,timedelta # To handle date and time
import pytz

In [ ]:
API_KEY = '6bd8c84137bbe5f4f19c61c2ab4e6da4' # Replace with youe actual API Key
BASE_URL = 'https://api.openweathermap.org/data/2.5' # Base url for making API request

1.   **Fetch Current Weather Data**

In [ ]:
def get_current_weather(city):
    url = f"{BASE_URL}/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()

    if response.status_code != 200 or 'main' not in data:
        print("Error fetching weather data:", data.get('message', 'Unknown error'))
        return None

    return {
        'city': data.get('name', city),
        'country': data.get('sys', {}).get('country', 'Unknown'),
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'wind_gust_dir': data['wind'].get('deg', 0),
        'pressure': data['main']['pressure'],
        'Wind_Gust_Speed': data['wind'].get('gust', data['wind'].get('speed', 0))
    }



2. Read Historical Data

In [ ]:
def read_historical_data(filename):
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)
    else:
        df = pd.read_excel(filename)  # Use for .xls or .xlsx

    df.columns = df.columns.str.strip()  # Clean column names
    print("✅ Columns:", df.columns.tolist())

    df = df.dropna()
    df = df[~df.duplicated()]
    return df

**Preparing Data for Training**

In [ ]:
def prepare_data(data):
    le = LabelEncoder()
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    x = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Temp', 'Pressure']]
    y = data['RainTomorrow']

    return x, y, le  # if you want to return the encoder

**Train Rain Prediction Model**

In [ ]:
def train_rain_model(x,y):
  x_train,x_test,y_train,y_test = train_test_split(x , y , random_state=4 ,test_size=0.2)
  model=RandomForestClassifier(n_estimators=100,random_state=4)
  model.fit(x_train,y_train)

  y_pred = model.predict(x_test)

  print("Square mean error for rain Model")

  print(mean_squared_error(y_test,y_pred))

  return model

**Prepare Regression Data**

In [ ]:
def prepare_regression_data(data, feature):
    x, y = [], []

    for i in range(len(data) - 1):
        x.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i + 1])

    x = np.array(x).reshape(-1, 1)
    y = np.array(y)

    print(f"🔎 Feature: {feature} | x shape: {x.shape}, y shape: {y.shape}")
    return x, y

**Train_regression_model**

In [ ]:
def train_regression_model(x,y):
  model = RandomForestRegressor(n_estimators=100 ,random_state=4)
  model.fit(x,y)
  return model


**Predict Future**

In [ ]:
def predict_future(model,current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])
  return predictions[1:]

**Weather Analysis Function**

In [ ]:
def weather_view():
    city = input("Enter any city name: ")

    current_weather = get_current_weather(city)

    # Load historical data
    historical_data = read_historical_data("WeatherRain_cleaned.csv")

    # Prepare and train rain prediction model
    x, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(x, y)

    # Map wind direction to compass points
    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75), ("N", 348.75, 360)
    ]

    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)[0]

    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather['Wind_Gust_Speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp']
    }

    current_df = pd.DataFrame([current_data], columns=['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Temp', 'Pressure'])

    # Rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

    # Regression models for temperature and humidity
    x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(x_temp, y_temp)
    hum_model = train_regression_model(x_hum, y_hum)

    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    # Prepare future time labels
    timezone = pytz.timezone('Asia/Kolkata')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_time = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # Display results
    print(f"\nCity: {city}, {current_weather.get('country', 'N/A')}")
    print(f"Current Temperature: {current_weather.get('current_temp', 'N/A')}°C")
    print(f"Feels Like: {current_weather.get('feels_like', 'N/A')}°C")
    print(f"Minimum Temperature: {current_weather.get('temp_min', 'N/A')}°C")
    print(f"Maximum Temperature: {current_weather.get('temp_max', 'N/A')}°C")
    print(f"Humidity: {current_weather.get('humidity', 'N/A')}%")
    print(f"Weather Description: {current_weather.get('description', 'N/A')}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_time, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Predictions:")
    for time, humidity in zip(future_time, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

# Call the function
weather_view()

Enter any city name: Datia
✅ Columns: ['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Temp', 'Pressure', 'RainTomorrow']
Square mean error for rain Model
0.07267676315081996
🔎 Feature: Temp | x shape: (55792, 1), y shape: (55792,)
🔎 Feature: Humidity | x shape: (55792, 1), y shape: (55792,)

City: Datia, IN
Current Temperature: 36°C
Feels Like: 41°C
Minimum Temperature: 36°C
Maximum Temperature: 36°C
Humidity: 46%
Weather Description: broken clouds
Rain Prediction: No

Future Temperature Predictions:
17:00: 35.6°C
18:00: 35.6°C
19:00: 35.6°C
20:00: 35.6°C
21:00: 35.6°C

Future Humidity Predictions:
17:00: 51.0%
18:00: 54.6%
19:00: 56.4%
20:00: 57.1%
21:00: 57.9%


In [ ]:
data = pd.read_csv('/content/WeatherRain_cleaned.csv')
data

,MinTemp,MaxTemp,WindGustDir,WindGustSpeed,Humidity,Temp,Pressure,RainTomorrow
0,13.4,22.9,W,44.0,71.0,22.0,1007.7,No
1,7.4,25.1,WNW,44.0,44.0,25.0,1010.6,No
2,12.9,25.7,WSW,46.0,38.0,30.0,1007.6,No
3,9.2,28.0,NE,24.0,45.0,16.0,1017.6,No
4,17.5,32.3,W,41.0,82.0,33.0,1010.8,No
...,...,...,...,...,...,...,...,...
63001,3.5,21.8,E,31.0,59.0,27.0,1024.7,No
63002,2.8,23.4,E,31.0,51.0,24.0,1024.6,No
63003,3.6,25.3,NNW,22.0,56.0,21.0,1023.5,No
63004,5.4,26.9,N,37.0,53.0,24.0,1021.0,No
